In [1]:
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import sounddevice as sd
import scipy
import pickle
import librosa
import pyloudnorm as pyln
%matplotlib qt5

import warnings
warnings.filterwarnings("ignore")


# VOWEL_GEN
This notebook encapsulates the functions in `lpc_synth.ipynb` to generate a vowel sound with various different parameters.

Main component:
- `gen_vowel()`: A function that uses `glottal_filter()` and `noise_filter()` to generate a specified vowel at a certain frequency.  Includes option for volume fluctuations (tremolo), and for adding breathiness to the vowel (air). Finally, normalizes the loudness of the vowel using `pyloudnorm`

In [25]:
# import lpc_synth notebook
%run lpc_synth.ipynb
fs = 44100

In [ ]:
# load the LPC coefficients we previously calculated
with open('lpc_coeffs/vowels2', 'rb') as file:
    lpc_coeffs = pickle.load(file)

# Generate a vowel v.
# 0.2 tremolo is a nice value
def gen_vowel(v, f0, T, slide = None, tremolo=0.25, air = 0.015, slide2 = None):
    lpc = lpc_coeffs[f3[1]][v]
    if v in vowels:
        lpc_air = lpc_coeffs[f"{v}_air"]
        y_air = noise_filter(lpc_air, T, magnitude=air)

    if slide is None:
        slide = FormantSlide(formant_thresh=1000, low_slide_start=0.9, high_slide_start=0.9)

    y = glottal_filter(lpc, f0, T, noise=0.012, slide=slide, slide2=slide2)

    brown_noise = gen_brown_noise(len(y))
    # upscale the noise so it is slower
    brown_noise = scipy.signal.resample(brown_noise, 3 * len(y))[0:len(y)] 
    brown_noise = brown_noise * tremolo + 1

    if v in vowels:
        y += y_air
    y *= brown_noise

    meter = pyln.Meter(fs, block_size=0.05)
    loudness = meter.integrated_loudness(y)
    y = pyln.normalize.loudness(y, loudness, -12.0)

    return y

In [ ]:
# y = gen_vowel('i', f0=200, T=2.0)
# play_audio(y)